In [1]:
import tensorflow
print(tensorflow.__file__)

C:\Users\mozoo\anaconda3\envs\tf\Lib\site-packages\tensorflow\__init__.py


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input


# -----------------------------
# 1) Load CIFAR-10
# -----------------------------
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

class_names = [
    "airplane","automobile","bird","cat","deer",
    "dog","frog","horse","ship","truck"
]

# Keep labels as integers (SparseCategoricalCrossentropy)
y_train = y_train.squeeze().astype("int64")
y_test  = y_test.squeeze().astype("int64")

# Convert images to float32
x_train = x_train.astype("float32")
x_test  = x_test.astype("float32")

# -----------------------------
# 2) Data augmentation
# -----------------------------
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
], name="augmentation")

# -----------------------------
# 3) Build MobileNetV2 backbone (pretrained)
# -----------------------------
mobile_base = MobileNetV2(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3)
)
mobile_base.trainable = False  # freeze first (feature extractor)

In [5]:


# -----------------------------
# 4) Full model (preprocess inside model)
# -----------------------------
mobilenet_model = keras.Sequential([
    layers.Input(shape=(32, 32, 3)),
    data_augmentation,
    layers.Resizing(224, 224, interpolation="bilinear"),
    layers.Lambda(preprocess_input),          # IMPORTANT: correct for ResNet50V2
    mobile_base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(10)                          # logits
], name="MobileNetV2")

mobilenet_model.summary()

# -----------------------------
# 5) Compile + Train (frozen backbone)
# -----------------------------
mobilenet_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=1),
]

history = mobilenet_model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=1,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)


Model: "MobileNetV2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ augmentation (Sequential)            │ (None, 32, 32, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ resizing_2 (Resizing)                │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda_2 (Lambda)                    │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │          12,810 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,270,794 (8.66 MB)

 Trainable params: 12,810 (50.04 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

704/704 ━━━━━━━━━━━━━━━━━━━━ 619s 865ms/step - accuracy: 0.6813 - loss: 0.9105 - val_accuracy: 0.8162 - val_loss: 0.5331 - learning_rate: 0.0010


In [6]:
# -----------------------------
# 6) Test / Evaluate
# -----------------------------
test_loss, test_acc_r = mobilenet_model.evaluate(x_test, y_test, verbose=0)
print("MobileNetV2 (frozen) test accuracy:", test_acc_r)
print("MobileNetV2 (frozen) test loss    :", test_loss)

MobileNetV2 (frozen) test accuracy: 0.8126000165939331
MobileNetV2 (frozen) test loss    : 0.5426377654075623


In [7]:
# -----------------------------
#Fine-tune last layers
# -----------------------------
mobile_base.trainable = True
mobilenet_model.trainable = True
for layer in mobile_base.layers[:-30]:
    layer.trainable = False

print("Trainable layers in backbone:", sum(l.trainable for l in mobile_base.layers), "/", len(mobile_base.layers))

mobilenet_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

history_ft = mobilenet_model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=3,
    batch_size=64,
    verbose=1
)

test_loss_ft, test_acc_ft = mobilenet_model.evaluate(x_test, y_test, verbose=0)
print("MobileNetV2 (fine-tuned) test accuracy:", test_acc_ft)
print("MobileNetV2 (fine-tuned) test loss    :", test_loss_ft)

Trainable layers in backbone: 30 / 154
Epoch 1/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 844s 1s/step - accuracy: 0.7125 - loss: 0.8256 - val_accuracy: 0.8222 - val_loss: 0.5170
Epoch 2/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 885s 1s/step - accuracy: 0.7741 - loss: 0.6513 - val_accuracy: 0.8408 - val_loss: 0.4511
Epoch 3/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 908s 1s/step - accuracy: 0.7976 - loss: 0.5823 - val_accuracy: 0.8510 - val_loss: 0.4179
MobileNetV2 (fine-tuned) test accuracy: 0.848800003528595
MobileNetV2 (fine-tuned) test loss    : 0.4394311010837555


In [8]:
# Collect and compare accuracies (update if you rename variables)
results = {
    "ResNet frozen test acc": float(test_acc_r) if 'test_acc_r' in globals() else None,
    "ResNet fine-tuned test acc": float(test_acc_ft) if 'test_acc_ft' in globals() else None,
}
for k,v in results.items():
    print(f"{k}: {v}")

ResNet frozen test acc: 0.8126000165939331
ResNet fine-tuned test acc: 0.848800003528595
